<a href="https://colab.research.google.com/github/danebencedavid/NLP-A-Agent/blob/master/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import faiss
import os
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import gradio as gr
import torch
from neo4j import GraphDatabase
from transformers import pipeline
from google.colab import userdata
from pathlib import Path

In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.4 MB/s eta 0:00:00


In [4]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.3 MB/s eta 0:00:00


In [5]:
pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 20.0 MB/s eta 0:00:00


In [7]:
uri = userdata.get('NEO4J_URI')
username = userdata.get('NEO4J_USER')
password = userdata.get('NEO4J_PASSWORD')

In [8]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
model = AutoModel.from_pretrained("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
embeddings_folder = "/content/drive/MyDrive/full_abs_embeddings"

In [10]:
full_df = pd.read_csv('/content/drive/MyDrive/final_second_wave.csv')

In [11]:
full_df['abstract'] = full_df['abstract'].fillna('')
non_empty_df = full_df[full_df['abstract'].str.strip() != ''].copy()
f_abstracts = non_empty_df['abstract'].tolist()
print(f"Number of non-empty abstracts: {len(f_abstracts)}")

Number of non-empty abstracts: 41


In [12]:
full_df = non_empty_df

In [13]:
full_df.columns

Index(['Unnamed: 0', 'cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid',
       'pubmed_id', 'license', 'abstract', 'publish_time', 'authors',
       'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files',
       'pmc_json_files', 'url', 's2_id', 'smoking_score', 'covid_score',
       'combined_score', 'diabetes_score', 'full_text', 'fulltext_path',
       'text_path'],
      dtype='object')

In [14]:
print(len(full_df))

41


In [28]:
embedding_files = sorted([f for f in os.listdir(embeddings_folder) if f.endswith(".npy")])
all_embeddings = [np.load(os.path.join(embeddings_folder, f)) for f in embedding_files]
embeddings = np.concatenate(all_embeddings, axis=0)
print(f"Loaded {embeddings.shape[0]} embeddings.")

Loaded 41 embeddings.


In [29]:
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [30]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

In [31]:
index.add(embeddings.astype(np.float32))

In [32]:
faiss_index_path = os.path.join(embeddings_folder, "faiss_index.index")
faiss.write_index(index, faiss_index_path)
print(f"FAISS index created and saved at {faiss_index_path}")

FAISS index created and saved at /content/drive/MyDrive/full_abs_embeddings/faiss_index.index


In [33]:
faiss_index_path = os.path.join(embeddings_folder, "faiss_index.index")
index = faiss.read_index(faiss_index_path)

In [19]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    return torch.sum(token_embeddings * input_mask_expanded, dim=1) / torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)

In [34]:
def embed_query(query, tokenizer, model, device):
    inputs = tokenizer([query], padding=True, truncation=True, return_tensors="pt", max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = mean_pooling(outputs, inputs["attention_mask"])

    embeddings = embeddings / embeddings.norm(p=2, dim=1, keepdim=True)

    return embeddings.cpu().numpy()


In [21]:
def read_full_text(path):
    if not path or not Path(path).exists():
        return ""

    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']

    for encoding in encodings:
        try:
            with open(path, 'r', encoding=encoding) as f:
                return f.read(15000)
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"Error reading {path}: {str(e)}")
            return ""

    try:
        with open(path, 'rb') as f:
            return str(f.read(15000))
    except Exception as e:
        print(f"Binary read failed for {path}: {str(e)}")
        return ""

In [22]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
qa_model_name = "google/flan-t5-large"
summary_model_name = "facebook/bart-large-cnn"

device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name).to(device)
qa_pipeline = pipeline(
    "text2text-generation",
    model=qa_model,
    tokenizer=qa_tokenizer,
    device=device
)

summary_tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
summary_model = AutoModelForSeq2SeqLM.from_pretrained(summary_model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [24]:
def search_top_k_with_full_text(query, index, df, k=3):
    try:
        query_embedding = embed_query(query, tokenizer, model, device)
        distances, indices = index.search(query_embedding, k)

        results = []
        for idx, dist in zip(indices[0], distances[0]):
            row = df.iloc[idx].copy()
            text_path = row.get("text_path", "")

            full_text = read_full_text(text_path) if text_path else row.get("abstract", "")
            if not full_text and 'abstract' in row:
                full_text = row['abstract']

            results.append({
                "cord_uid": row.get("cord_uid", ""),
                "title": row.get("title", "No title"),
                "distance": float(dist),
                "full_text": str(full_text)[:15000],
                "source": "fulltext" if text_path and full_text else "abstract",
                "doi": row.get("doi","")
            })

        return pd.DataFrame(results)

    except Exception as e:
        print(f"Search error: {str(e)}")
        return pd.DataFrame(columns=["cord_uid", "title", "distance", "full_text", "source",'doi'])

In [25]:
def summarize_documents(documents, max_length=300):
    try:
        inputs = summary_tokenizer(
            ["summarize: " + doc[:25000] for doc in documents],
            max_length=1024,
            truncation=True,
            padding=True,
            return_tensors="pt"
        ).to(device)

        summary_ids = summary_model.generate(
            inputs["input_ids"],
            max_length=max_length,
            min_length=30,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        return summary_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    except Exception as e:
        print(f"Summarization error: {str(e)}")
        return ["Summary unavailable"] * len(documents)

In [26]:
def generate_answer(query, context):
    try:
        prompt = f"""Answer this question in a 5-6 sentences based on the context:

        Question: {query}
        Context: {context}

        Answer:"""

        return qa_pipeline(
            prompt,
            max_new_tokens=300,
            do_sample=False,
            temperature=0.3,
            truncation=True
        )[0]["generated_text"]
    except Exception as e:
        print(f"QA error: {str(e)}")
        return "Could not generate answer"

In [35]:
query = "How does smoking effect COVID-19 ?"

In [36]:
search_results_df = search_top_k_with_full_text(query, index, full_df, k=3)
search_results_df

,cord_uid,title,distance,full_text,source,doi
0,34hc3ulc,Renin-Angiotensin-System (RAS) und COVID-19 – ...,0.478589,Renin-Angiotensin-System (RAS) und COVID-19\nZ...,fulltext,10.1055/a-1152-3469
1,5xcil8eb,Trilogy of ACE2: A peptidase in the renin–angi...,0.431758,Associate editor: M. Endoh\nTrilogy of ACE2: A...,fulltext,10.1016/j.pharmthera.2010.06.003
2,h0h6cri8,Pharmacologic modulation of ACE2 expression,0.424537,Pharmacologic Modulation \nof ACE2 Expression...,fulltext,10.1007/s11906-008-0076-0


In [37]:
top_cited_dois = search_results_df['doi'].tolist()

In [38]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [39]:
citing_papers_from_neo4j = []

In [41]:
with driver.session() as session:
    def get_papers_citing_these_dois(tx, cited_dois):
        query = """
        MATCH (c:Paper)-[:CITES]->(p:Paper)
        WHERE p.doi IN $cited_dois
        RETURN DISTINCT c.doi, c.title, c.abstract, c.abstract_embedding
        """
        result = tx.run(query, cited_dois=cited_dois)
        citing_papers = [{"doi": record["c.doi"], "title": record["c.title"], "abstract": record["c.abstract"], "vector": record.get("c.abstract_embedding")} for record in result]
        return citing_papers

    citing_papers_from_neo4j = session.execute_read(get_papers_citing_these_dois, top_cited_dois)

driver.close()

print(" citing:", citing_papers_from_neo4j)

 citing: [{'doi': '10.1155/2023/7890188', 'title': 'Angiotensin-Converting Enzyme and Hypertension: A Systemic Analysis of Various ACE Inhibitors, Their Side Effects, and Bioactive Peptides as a Putative Therapy for Hypertension', 'abstract': 'Hypertension is a major risk factor for heart attack, produce atherosclerosis (hardening of the arteries), congestive heart failure, stroke, kidney infection, blindness, end-stage renal infection, and cardiovascular diseases. Many mechanisms are involved in causing hypertension, i.e., via calcium channels, alpha and beta receptors, and the renin-angiotensin system (RAS). RAS has an important role in blood pressure control and is also involved in the metabolism of glucose, homeostasis, and balance of electrolytes in the body. The components of RAS that are involved in the regulation of blood pressure are angiotensinogen, Ang I (angiotensin I), Ang II (angiotensin II), ACE (angiotensin-converting enzyme), and ACE 2 (angiotensin-converting enzyme 2)

In [42]:
def calculate_similarity(embedding1, embedding2):
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

In [43]:
query_embedding = embed_query(query, tokenizer, model, device)

In [44]:
original_similarities = []
for index, row in search_results_df.iterrows():
    abstract = row.get('abstract')
    if abstract:
        original_embedding = embed_texts([abstract])[0]
        similarity = calculate_similarity(query_embedding, original_embedding)
        original_similarities.append({'doi': row.get('doi'), 'similarity': similarity, 'source': 'original'})
    else:
        original_similarities.append({'doi': row.get('doi'), 'similarity': -1, 'source': 'original'})

In [45]:
citing_similarities = []
for paper in citing_papers_from_neo4j:
    vector = paper.get('vector')
    if vector is not None:
        similarity = calculate_similarity(query_embedding, vector)
        citing_similarities.append({'doi': paper.get('doi'), 'similarity': similarity, 'source': 'citing', 'paper': paper})
    else:
        citing_similarities.append({'doi': paper.get('doi'), 'similarity': -1, 'source': 'citing', 'paper': paper})

best_paper = None
best_similarity = -float('inf')

In [46]:
for item in original_similarities:
    if item['similarity'] > best_similarity:
        best_similarity = item['similarity']
        best_paper = {'doi': item['doi'], 'source': item['source']}

for item in citing_similarities:
    if item['similarity'] > best_similarity:
        best_similarity = item['similarity']
        best_paper = {'doi': item['doi'], 'source': item['source'], 'paper_data': item['paper']}

print("Best paper based on similarity:")
if best_paper:
    print(f"Source: {best_paper['source']}")
    print(f"DOI: {best_paper['doi']}")
    if 'paper_data' in best_paper:
        print("Title:", best_paper['paper_data'].get('title'))
        print("Abstract (if available):", best_paper['paper_data'].get('abstract', 'N/A')[:200] + "...")
    print(f"Similarity: {best_similarity}")
else:
    print("No relevant papers found.")

Best paper based on similarity:
Source: citing
DOI: 10.1111/jch.70021
Title: Antihypertensive Drug Use and COVID‐19 Disease Severity in Hospitalized US Veterans: A Retrospective Cohort Study
Abstract (if available): This research investigated associations between hypertensive patients with COVID-19 who did and did not use specific antihypertensive medications and had incurred hospital ventilation/death in the US ...
Similarity: 0.4006262761993381


In [47]:
def get_content_for_answer_smart(best_paper, full_df, summarize_documents_func):
    content = ""
    should_summarize = False

    if best_paper:
        if 'paper_data' in best_paper:
            paper = best_paper['paper_data']
            full_text = paper.get('full_text')
            if full_text and isinstance(full_text, str) and len(full_text) > 1500:
                content = full_text
                should_summarize = True
            elif full_text and isinstance(full_text, str):
                content = full_text
            else:
                abstract = paper.get('abstract')
                if abstract and isinstance(abstract, str) and abstract.strip():
                    content = abstract
                else:
                    title = paper.get('title', "No title")
                    content = title
        elif best_paper['source'] == 'original':
            doi = best_paper['doi']
            original_row = full_df[full_df['doi'] == doi].iloc[0]
            text_path = original_row.get('text_path')
            if text_path and Path(text_path).exists():
                full_text = read_full_text(text_path)
                if len(full_text) > 1500:
                    content = full_text
                    should_summarize = True
                else:
                    content = full_text
            elif 'abstract' in original_row and isinstance(original_row['abstract'], str) and original_row['abstract'].strip():
                content = original_row['abstract']
            else:
                content = original_row.get('title', "No title")
    else:
        return "No relevant content found."

    if should_summarize and content:
        return summarize_documents_func([content])[0]
    else:
        return content

In [48]:
content_for_answer = get_content_for_answer_smart(best_paper, full_df, summarize_documents)

In [49]:
content_for_answer

'This research investigated associations between hypertensive patients with COVID-19 who did and did not use specific antihypertensive medications and had incurred hospital ventilation/death in the US Veterans Hospital System. This study included de-identified medical records of 572 994 hypertensive US veterans who were diagnosed with COVID-19 and hospitalized between March 1, 2020, and February 28, 2022, with a 60-day follow-up period. Mechanical ventilation and/or death within 60 days of COVID-19 hospitalization served as the outcome variables. On multivariable analysis, CCB use was associated with a 9% increased risk of mechanical ventilation, while ACE inhibitors (HR: 0.90), alpha blockers (HR: 0.92), and CCB (HR: 0.93) users were associated with a significantly decreased risk of death. Additional multivariable analysis on those with and without additional comorbidities, such as chronic kidney disease (CKD), heart failure, and diabetes, revealed that in those with at least one addi

In [50]:
answer = generate_answer(query, content_for_answer)
print("Answer:\n", answer)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer:
 CCB use was associated with a 9% increased risk of mechanical ventilation, while ACE inhibitors (HR: 0.90), alpha blockers (HR: 0.92), and CCB (HR: 0.93) users were associated with a significantly decreased risk of death.


In [53]:
def full_pipeline(query):
    global index

    if not isinstance(globals().get("index", None), faiss.Index):
      index = faiss.read_index(faiss_index_path)
    print(f"FAISS index loaded: {type(index)}")


    try:
        print(f"\n=== Processing query: '{query}' ===")

        # Step 1: Search for relevant papers
        print("1. Searching papers...")
        try:
            print(f"Debug - Checking variables:")
            print(f"- index type: {type(index)}")
            print(f"- full_df shape: {full_df.shape}")
            print(f"- device: {device}")

            print("\nSample data from full_df:")
            print(full_df[['doi', 'title']].head(2))
            search_results_df = search_top_k_with_full_text(
                query=query,
                index=index,
                df=full_df,
                k=3
            )

            if search_results_df.empty:
                print("Warning: Empty search results!")
                return "No relevant papers found for your query."

            print(f"Found {len(search_results_df)} search results")
            print("\nTop search results:")
            print(search_results_df[['doi', 'title']].head(3))

        except Exception as e:
            print(f"Search failed: {type(e).__name__}: {str(e)}")
            return f"Search error: {str(e)}"

        # Step 2: Query Neo4j for citing papers
        print("\n2. Querying Neo4j for citing papers...")
        top_cited_dois = [str(doi) for doi in search_results_df['doi'].tolist() if pd.notna(doi)]
        print(f"DOIs being queried: {top_cited_dois}")

        citing_papers_from_neo4j = []
        driver = None
        try:
            driver = GraphDatabase.driver(uri, auth=(username, password))
            with driver.session() as session:
                citing_papers_from_neo4j = session.execute_read(get_papers_citing_these_dois, top_cited_dois)
            print(f"Found {len(citing_papers_from_neo4j)} citing papers")

            if citing_papers_from_neo4j:
                print("\nSample citing papers:")
                for paper in citing_papers_from_neo4j[:3]:
                    print(f"- {paper.get('title', 'No title')} (DOI: {paper.get('doi', 'N/A')})")
        except Exception as neo4j_error:
            print(f"Neo4j error: {str(neo4j_error)}")
            citing_papers_from_neo4j = []
        finally:
            if driver:
                driver.close()

        # Step 3: Calculate similarities
        print("\n3. Calculating similarities...")
        try:
            query_embedding = embed_query(query, tokenizer, model, device)
            print("Query embedding generated successfully")
        except Exception as e:
            print(f"Embedding generation failed: {str(e)}")
            return f"Embedding error: {str(e)}"

        original_similarities = []
        for i, row in search_results_df.iterrows():
            try:
                abstract = row.get('abstract', row.get('title', ''))
                if abstract:
                    original_embedding = embed_query(abstract, tokenizer, model, device)
                    similarity = calculate_similarity(query_embedding, original_embedding)
                    original_similarities.append({
                        'doi': row.get('doi'),
                        'similarity': similarity,
                        'source': 'original',
                        'title': row.get('title', 'No title'),
                        'abstract': abstract
                    })
                    print(f"Similarity with {row.get('doi', 'N/A')}: {similarity:.4f}")
            except Exception as e:
                print(f"Similarity calc failed for {row.get('doi', 'N/A')}: {str(e)}")

        citing_similarities = []
        for paper in citing_papers_from_neo4j:
            try:
                vector = paper.get('vector')
                abstract = paper.get('abstract', paper.get('title', ''))
                if vector is not None:
                    similarity = calculate_similarity(query_embedding, vector)
                    citing_similarities.append({
                        'doi': paper.get('doi'),
                        'similarity': similarity,
                        'source': 'citing',
                        'title': paper.get('title', 'No title'),
                        'abstract': abstract
                    })
                    print(f"Similarity with citing paper {paper.get('doi', 'N/A')}: {similarity:.4f}")
                elif abstract:
                    citing_embedding = embed_query(abstract, tokenizer, model, device)
                    similarity = calculate_similarity(query_embedding, citing_embedding)
                    citing_similarities.append({
                        'doi': paper.get('doi'),
                        'similarity': similarity,
                        'source': 'citing',
                        'title': paper.get('title', 'No title'),
                        'abstract': abstract
                    })
                    print(f"Similarity (fallback) with {paper.get('doi', 'N/A')}: {similarity:.4f}")
            except Exception as e:
                print(f"Similarity calc failed for citing paper {paper.get('doi', 'N/A')}: {str(e)}")

        # Step 4: Find best match
        print("\n4. Determining best match...")
        all_results = original_similarities + citing_similarities
        if not all_results:
            print("Warning: No valid similarity results!")
            return "No relevant content found to answer your question."

        best_match = max(all_results, key=lambda x: x['similarity'])
        print(f"Best match: {best_match['title']} (Similarity: {best_match['similarity']:.4f})")

        # Step 5: Retrieve content
        print("\n5. Retrieving content...")
        content = ""
        try:
            if best_match['source'] == 'original':
                paper_row = full_df[full_df['doi'] == best_match['doi']].iloc[0]
                text_path = paper_row.get('text_path')
                if text_path and Path(text_path).exists():
                    content = read_full_text(text_path)
                    print("Using full text from path")
                else:
                    content = paper_row.get('abstract', paper_row.get('title', ''))
                    print("Using abstract/title")
            else:
                content = best_match.get('abstract', best_match.get('title', ''))
                print("Using citing paper content")

            print(f"Content length: {len(content)} characters")
        except Exception as e:
            print(f"Content retrieval failed: {str(e)}")
            return f"Content error: {str(e)}"

        # Step 6: Summarize if needed
        if len(content) > 1500:
            print("\n6. Summarizing long content...")
            try:
                content = summarize_documents([content])[0]
                print(f"Summarized to {len(content)} characters")
            except Exception as e:
                print(f"Summarization failed: {str(e)}")
                content = content[:1500]

        # Step 7: Generate answer
        print("\n7. Generating answer...")
        try:
            answer = generate_answer(query, content)
            print("Answer generated successfully")
        except Exception as e:
            print(f"Answer generation failed: {str(e)}")
            return f"Answer generation error: {str(e)}"

        # Prepare output
        output = f"Answer:\n{answer}\n\n"
        output += f"Source:\nTitle: {best_match['title']}\n"
        if best_match['source'] == 'original':
            output += f"(Original paper from search results)"
        else:
            output += f"(Citing paper from knowledge graph)"

        if best_match.get('doi'):
            output += f"\nDOI: {best_match['doi']}"

        return output

    except Exception as e:
        print(f"\n!!! Pipeline crashed: {type(e).__name__}: {str(e)}")
        return f"System error: {str(e)}"

# Enhanced Gradio interface with debug mode
with gr.Blocks(title="Research Paper QA System") as demo:
    gr.Markdown("""
    # Biomedical Research Question Answering System
    Ask questions about COVID-19 and related biomedical research.
    The system will search through research papers and knowledge graphs to find answers.
    """)

    with gr.Row():
        question_input = gr.Textbox(
            label="Your question",
            placeholder="e.g., How does smoking impact COVID-19 outcomes?",
            lines=3
        )

    with gr.Row():
        submit_btn = gr.Button("Get Answer", variant="primary")
        debug_toggle = gr.Checkbox(label="Show debug logs")

    with gr.Accordion("Debug Output", open=False):
        debug_output = gr.Textbox(label="Console Logs", interactive=False, lines=10)

    output_area = gr.Textbox(
        label="Answer",
        interactive=False,
        lines=10
    )

    examples = gr.Examples(
        examples=[
            "How does smoking impact the lungs' response to COVID-19 infection?",
            "What are the effects of remdesivir on COVID-19 mortality?",
            "How does age affect COVID-19 severity?",
            "What is the relationship between ACE2 receptors and COVID-19?"
        ],
        inputs=question_input
    )

    def wrapped_pipeline(query, debug):
        import sys
        from io import StringIO

        if debug:
            old_stdout = sys.stdout
            sys.stdout = mystdout = StringIO()

            result = full_pipeline(query)

            sys.stdout = old_stdout
            debug_info = mystdout.getvalue()

            return result, debug_info
        else:
            result = full_pipeline(query)
            return result, ""

    submit_btn.click(
        fn=wrapped_pipeline,
        inputs=[question_input, debug_toggle],
        outputs=[output_area, debug_output]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af5dac2f7f9250650d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
